## Создание таблицы факторов по кукам для кред скоринга

In [ ]:
import datetime

In [ ]:
try:
    sc.stop()
except:
    pass

In [ ]:
create_accum_pattern = '''

drop table if exists user_kposminin.id_feat_accum;
CREATE TABLE `user_kposminin.id_feat_accum`(
	  `id` string, 
	  `load_src` string, 
	  `first_id_ymd` string, 
	  `last_id_ymd` string, 
	  `ymd_cnt` int, 
	  `cnt` bigint, 
	  `visits_cnt` bigint, 
	  `hits` bigint, 
	  `emailru` bigint, 
	  `mobile_share` double, 
	  `vk_share` double, 
	  `social_share` double, 
	  `work_hours_hits_share` double, 
	  `avg_hour_sum_sq` bigint, 
	  `avg_hour_sum` bigint, 
	  `max_score1` double, 
	  `max_score2` double, 
	  `max_score3` double, 
	  `max_score4` double, 
	  `min_score1` double, 
	  `min_score2` double, 
	  `min_score3` double, 
	  `min_score4` double, 
	  `sum_score1` double, 
	  `sum_score2` double, 
	  `sum_score3` double, 
	  `sum_score4` double, 
	  `good_urlfr_share_score1` double, 
	  `good_urlfr_share_score2` double, 
	  `good_urlfr_share_score3` double,
	  `first_calc_ymd` string, 
	  `last_calc_ymd` string   
      )
;

'''

create_accum_store_pattern = '''
CREATE TABLE `user_kposminin.id_feat_accum_store`(
	  `id` string, 
	  `load_src` string, 
	  `first_id_ymd` string, 
	  `last_id_ymd` string, 
	  `ymd_cnt` int, 
	  `cnt` bigint, 
	  `visits_cnt` bigint, 
	  `hits` bigint, 
	  `emailru` bigint, 
	  `mobile_share` double, 
	  `vk_share` double, 
	  `social_share` double, 
	  `work_hours_hits_share` double, 
	  `avg_hour_sum_sq` bigint, 
	  `avg_hour_sum` bigint, 
	  `max_score1` double, 
	  `max_score2` double, 
	  `max_score3` double, 
	  `max_score4` double, 
	  `min_score1` double, 
	  `min_score2` double, 
	  `min_score3` double, 
	  `min_score4` double, 
	  `sum_score1` double, 
	  `sum_score2` double, 
	  `sum_score3` double, 
	  `sum_score4` double, 
	  `good_urlfr_share_score1` double, 
	  `good_urlfr_share_score2` double, 
	  `good_urlfr_share_score3` double)
partitioned by (
	  `first_calc_ymd` string, 
	  `last_calc_ymd` string      
      )
;

'''

feat_1d_query_pattern = '''

-- Факторы по кукам. #ymd
create table user_kposminin.id_feat_1d_#ind as
with mymd as 
 (select
   target,
   max(ymd) as max_ymd
  from
   user_kposminin.urlfr_tgt_cnt
  where 
    ymd < date_add('#ymd',-3)
  group by target
 )

select 
 id,
 load_src, 
 count(distinct urlfr) as cnt,
 count(*) as visits_cnt,
 sum(cnt) as hits,
 sum(if(urlfr like 'e.mail.ru%',1,0)) as emailru,
 sum(if(urlfr like 'm.%',1,0))/sum(1) as mobile_share,
 sum(if(urlfr rlike '^(m\\.)?vk.com%', 1, 0))/sum(1) as vk_share,
 sum(if(urlfr like 'vk.com%' or urlfr rlike '^(m\\.)?ok\\.ru' or urlfr like 'm.odnoklassniki.ru%' or urlfr rlike '^(m\\.)?my.mail.ru',1,0))/sum(1) as social_share,
 sum(if(avg_hour >= 9 and avg_hour <= 20,cnt,0))/sum(cnt) as work_hours_hits_share,
 sum( avg_hour * avg_hour) as avg_hour_sum_sq,
 sum(avg_hour) as avg_hour_sum,
 max(score1) as max_score1,
 max(score2) as max_score2,
 max(score3) as max_score3,
 max(score4) as max_score4,
 min(score1) as min_score1,
 min(score2) as min_score2,
 min(score3) as min_score3,
 min(score4) as min_score4,
 sum(score1) as sum_score1,
 sum(score2) as sum_score2,
 sum(score3) as sum_score3,
 sum(score4) as sum_score4, 
 count( if(score1 > 1, urlfr,Null))/sum(1) as good_urlfr_share_score1,
 count( if(score2 > -7, urlfr,Null))/sum(1) as good_urlfr_share_score2,
 count( if(score3 > -7, urlfr,Null))/sum(1) as good_urlfr_share_score3,
 ymd
 
from
 (select
    v.id,
    v.load_src,
    v.ymd,
    v.url_fragment as urlfr,
    unix_timestamp(v.ymd, 'yyyy-MM-dd')/60/60 + v.average_visit_hour  as time_h,
    1 as time_std,
    v.visit_count as cnt,
    v.average_visit_hour as avg_hour,
    t1.score as score1,
    t2.score as score2,
    t3.score as score3,
    t4.score as score4
  from
    prod_odd.visit_feature v
    left semi join (select 
       uid_str as id,
       property_value as phone_num,
       load_src
     from
       prod_dds.md_uid_property 
     where
       property_cd = 'PHONE'
    ) m on m.id = v.id and m.load_src = v.load_src  
    left join (
        select urlfr,score
          from mymd td
         inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
         where td.target = 'ccall_tinkoff_approve_from_fullapp'
    ) t1 on t1.urlfr = v.url_fragment
    left join (
        select urlfr,score
          from mymd td
         inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
         where td.target = 'tinkoff_platinum_approved_application03@tinkoff_action' 
    ) t2 on t2.urlfr = v.url_fragment
    left join (
        select urlfr,score
          from mymd td
         inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
         where td.target = 'tinkoff_platinum_complete_application03@tinkoff_action'
    ) t3 on t3.urlfr = v.url_fragment
    left join (
        select urlfr,score
          from mymd td
         inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
         where td.target = 'tinkoff_LON_CCR_default'
    ) t4 on t4.urlfr = v.url_fragment
  where 
    v.ymd = '#ymd' 
 ) a 
group by
  id,load_src,ymd
;
'''

accumulator_update_pattern = '''

create table user_kposminin.id_feat_accum_upd as
select
  nvl(a.id,b.id) as id,
  nvl(a.load_src,b.load_src) as load_src,
  least(a.first_id_ymd,b.ymd) as first_id_ymd,
  greatest(a.last_id_ymd,b.ymd) as last_id_ymd,
  nvl(a.ymd_cnt,0) + if(b.ymd is Null,0,1) as ymd_cnt,
  nvl(a.cnt,0) + nvl(b.cnt,0) as cnt,
  nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0) as visits_cnt,
  nvl(a.hits,0) + nvl(b.hits,0) as hits,
  nvl(a.emailru,0) + nvl(b.emailru,0) as emailru,
  (nvl(a.mobile_share * a.visits_cnt,0) + nvl(b.mobile_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as mobile_share,
  (nvl(a.vk_share * a.visits_cnt,0) + nvl(b.vk_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as vk_share,
  (nvl(a.social_share * a.visits_cnt,0) + nvl(b.social_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as social_share,
  (nvl(a.work_hours_hits_share * a.visits_cnt,0) + nvl(b.work_hours_hits_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as work_hours_hits_share,
  nvl(a.avg_hour_sum_sq,0) + nvl(b.avg_hour_sum_sq,0) as avg_hour_sum_sq,
  nvl(a.avg_hour_sum,0) + nvl(b.avg_hour_sum,0) as avg_hour_sum,
  greatest(a.max_score1,b.max_score1) as max_score1,
  greatest(a.max_score2,b.max_score2) as max_score2,
  greatest(a.max_score3,b.max_score3) as max_score3,
  greatest(a.max_score4,b.max_score4) as max_score4,
  least(a.max_score1,b.max_score1) as min_score1,
  least(a.max_score2,b.max_score2) as min_score2,
  least(a.max_score3,b.max_score3) as min_score3,
  least(a.max_score4,b.max_score4) as min_score4,
  nvl(a.sum_score1,0) + nvl(b.sum_score1,0) as sum_score1,
  nvl(a.sum_score2,0) + nvl(b.sum_score2,0) as sum_score2,
  nvl(a.sum_score3,0) + nvl(b.sum_score3,0) as sum_score3,
  nvl(a.sum_score4,0) + nvl(b.sum_score4,0) as sum_score4,
  (nvl(a.good_urlfr_share_score1 * a.visits_cnt,0) + nvl(b.good_urlfr_share_score1 * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as good_urlfr_share_score1,
  (nvl(a.good_urlfr_share_score2 * a.visits_cnt,0) + nvl(b.good_urlfr_share_score2 * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as good_urlfr_share_score2,
  (nvl(a.good_urlfr_share_score3 * a.visits_cnt,0) + nvl(b.good_urlfr_share_score3 * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as good_urlfr_share_score3,
  '#first_ymd' as first_calc_ymd,
  '#ymd' as last_calc_ymd
from 
  user_kposminin.id_feat_accum a
  full join user_kposminin.id_feat_1d_#ind b on a.id = b.id and a.load_src = b.load_src
;

drop table user_kposminin.id_feat_accum;
alter table user_kposminin.id_feat_accum_upd rename to user_kposminin.id_feat_accum;

'''

write_acc_query = '''
insert overwrite table user_kposminin.id_feat_accum_store partition (first_calc_ymd,last_calc_ymd)
select * from user_kposminin.id_feat_accum;

'''

In [ ]:
day = datetime.datetime(2017,1,22)

gen_1d_query,acc_upd_query = '', ''

for _ in range(92):
    day = day + datetime.timedelta(days = 1)
    gen_1d_query += feat_1d_query_pattern.replace('#ymd',day.strftime('%Y-%m-%d')).replace('#ind',day.strftime('%Y%m%d'))
    acc_upd_query += accumulator_update_pattern.replace('#ymd',day.strftime('%Y-%m-%d')).replace('#ind',day.strftime('%Y%m%d'))

#print(gen_1d_query + '\n\n' + acc_upd_query)

### Генерация полумесячных батчей

In [ ]:
interval = 15
acc_upd_query = ''
start_dates = [datetime.datetime(2017, 2, 24) - datetime.timedelta(days = interval * i) for i in range( 60/interval) ]
for first_ymd in start_dates:
    acc_upd_query += create_accum_pattern
    for i in range(60 + interval):
        day = first_ymd + datetime.timedelta(days = i)
        acc_upd_query += (accumulator_update_pattern
                            .replace('#ymd',day.strftime('%Y-%m-%d'))
                            .replace('#ind',day.strftime('%Y%m%d'))
                            .replace('#first_ymd',first_ymd.strftime('%Y-%m-%d'))
                         )
        if (i > 60):
            acc_upd_query += (write_acc_query
                                .replace('#ymd',day.strftime('%Y-%m-%d'))
                                .replace('#ind',day.strftime('%Y%m%d'))
                                .replace('#first_ymd',first_ymd.strftime('%Y-%m-%d'))
                             )
        
print(acc_upd_query)

In [ ]:
def main():
    interval = 15
    acc_upd_query = ''
    start_dates = reduce(lambda x,y:x+y,[[datetime.datetime(2017, i, 1),datetime.datetime(2017, i, 16)]  for i in range(1, 6)])
    for first_ymd,next_ymd in zip(start_dates[:-1],start_dates[1:]):
        day = first_ymd
        acc_upd_query += create_accum_pattern
        while day < next_ymd:            
            acc_upd_query += (accumulator_update_pattern
                            .replace('#ymd',day.strftime('%Y-%m-%d'))
                            .replace('#ind',day.strftime('%Y%m%d'))
                            .replace('#first_ymd',first_ymd.strftime('%Y-%m-%d'))
                             )
            day = day + datetime.timedelta(days = 1)
        acc_upd_query += (write_acc_query
                                .replace('#ymd',day.strftime('%Y-%m-%d'))
                                .replace('#ind',day.strftime('%Y%m%d'))
                                .replace('#first_ymd',first_ymd.strftime('%Y-%m-%d'))
                                 )        
    #exec_queries(cursor,acc_upd_query)
    print(acc_upd_query)
    print('End')


In [ ]:
dates = [datetime.datetime(2017, 4, 25).date() + datetime.timedelta(days = i) for i in range(0, 8)]
query = '''insert overwrite table user_kposminin.id_feat_accum 
select * from user_kposminin.id_feat_accum_store where first_calc_ymd = '2017-02-24' and last_calc_ymd = '2017-04-24' ; '''

for day in dates:
    query += (accumulator_update_pattern
                            .replace('#ymd',day.strftime('%Y-%m-%d'))
                            .replace('#ind',day.strftime('%Y%m%d'))
                            .replace('#first_ymd','2017-02-24')
             )
    query += (write_acc_query
                                .replace('#ymd',day.strftime('%Y-%m-%d'))
                                .replace('#ind',day.strftime('%Y%m%d'))
                                .replace('#first_ymd','2017-02-24')
             )
print(query)

### Расчет скоринговых фич за март- начало апреля 2016 с использованием полумесячных батчей.

In [ ]:
a = '''01.01.2017	15.01.2017
16.01.2017	31.01.2017
01.02.2017	15.02.2017
16.02.2017	28.02.2017
01.03.2017	15.03.2017
16.03.2017	31.03.2017
01.04.2017	15.04.2017
'''

In [ ]:
dates = [[datetime.datetime.strptime(v,'%d.%m.%Y').date() for v in e.split('\t')] for e in a.split('\n')[:-1]]

In [ ]:
accumulator_merge_init_query = '''
insert overwrite table user_kposminin.id_feat_accum 
select * from user_kposminin.id_feat_accum_store 
where first_calc_ymd = '#first_calc_ymd' and last_calc_ymd = '#last_calc_ymd' 
;
'''


accumulator_merge_pattern = '''

create table user_kposminin.id_feat_accum_upd as
select
  nvl(a.id,b.id) as id,
  nvl(a.load_src,b.load_src) as load_src,
  least(a.first_id_ymd,b.first_id_ymd) as first_id_ymd,
  greatest(a.last_id_ymd,b.last_id_ymd) as last_id_ymd,
  nvl(a.ymd_cnt,0) + nvl(b.ymd_cnt,0) as ymd_cnt,
  nvl(a.cnt,0) + nvl(b.cnt,0) as cnt,
  nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0) as visits_cnt,
  nvl(a.hits,0) + nvl(b.hits,0) as hits,
  nvl(a.emailru,0) + nvl(b.emailru,0) as emailru,
  (nvl(a.mobile_share * a.visits_cnt,0) + nvl(b.mobile_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as mobile_share,
  (nvl(a.vk_share * a.visits_cnt,0) + nvl(b.vk_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as vk_share,
  (nvl(a.social_share * a.visits_cnt,0) + nvl(b.social_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as social_share,
  (nvl(a.work_hours_hits_share * a.visits_cnt,0) + nvl(b.work_hours_hits_share * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as work_hours_hits_share,
  nvl(a.avg_hour_sum_sq,0) + nvl(b.avg_hour_sum_sq,0) as avg_hour_sum_sq,
  nvl(a.avg_hour_sum,0) + nvl(b.avg_hour_sum,0) as avg_hour_sum,
  greatest(a.max_score1,b.max_score1) as max_score1,
  greatest(a.max_score2,b.max_score2) as max_score2,
  greatest(a.max_score3,b.max_score3) as max_score3,
  greatest(a.max_score4,b.max_score4) as max_score4,
  least(a.max_score1,b.max_score1) as min_score1,
  least(a.max_score2,b.max_score2) as min_score2,
  least(a.max_score3,b.max_score3) as min_score3,
  least(a.max_score4,b.max_score4) as min_score4,
  nvl(a.sum_score1,0) + nvl(b.sum_score1,0) as sum_score1,
  nvl(a.sum_score2,0) + nvl(b.sum_score2,0) as sum_score2,
  nvl(a.sum_score3,0) + nvl(b.sum_score3,0) as sum_score3,
  nvl(a.sum_score4,0) + nvl(b.sum_score4,0) as sum_score4,
  (nvl(a.good_urlfr_share_score1 * a.visits_cnt,0) + nvl(b.good_urlfr_share_score1 * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as good_urlfr_share_score1,
  (nvl(a.good_urlfr_share_score2 * a.visits_cnt,0) + nvl(b.good_urlfr_share_score2 * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as good_urlfr_share_score2,
  (nvl(a.good_urlfr_share_score3 * a.visits_cnt,0) + nvl(b.good_urlfr_share_score3 * b.visits_cnt,0)) / (nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0)) as good_urlfr_share_score3,
  least(a.first_calc_ymd, b.first_calc_ymd) as first_calc_ymd,
  greatest(a.last_calc_ymd, b.last_calc_ymd) as last_calc_ymd
from 
  user_kposminin.id_feat_accum a
  full join user_kposminin.id_feat_accum_store b on a.id = b.id and a.load_src = b.load_src
  where b.first_calc_ymd = '#first_calc_ymd' and b.last_calc_ymd = '#last_calc_ymd' 
;

drop table user_kposminin.id_feat_accum;
alter table user_kposminin.id_feat_accum_upd rename to user_kposminin.id_feat_accum;

'''

In [ ]:
query = ''
for i in range(len(dates)-4):
    query += (accumulator_merge_init_query
              .replace('#first_calc_ymd',dates[i][0].strftime('%Y-%m-%d'))
              .replace('#last_calc_ymd',dates[i][1].strftime('%Y-%m-%d'))
             )
    for j in range(1,4):
        query += (accumulator_merge_pattern
                   .replace('#first_calc_ymd',dates[i+j][0].strftime('%Y-%m-%d'))
                   .replace('#last_calc_ymd',dates[i+j][1].strftime('%Y-%m-%d'))
                 )
    day = dates[i+3][1]
    while day < dates[i+4][1]:
        query += (write_acc_query
                 )
        day += datetime.timedelta(days = 1)
        query += (accumulator_update_pattern
                            .replace('#ymd',day.strftime('%Y-%m-%d'))
                            .replace('#ind',day.strftime('%Y%m%d'))
                            .replace('#first_ymd',dates[i][0].strftime('%Y-%m-%d'))
                 )
    query += (write_acc_query
                 )
print(query)

In [ ]:
### Тестирование. скоринг полных заявок по холодному обзвону.

In [ ]:
sas_query = '''
proc sql;
create table hd_ccall.full_app_201704 as 
select
  put(datepart(create_dt),YYMMDD10.) as ymd,
  b.phone_mobile,
  b.utm_campaign,
  b.hl_rk,  
  case when status = 'В работе' then 1 else 0 end as in_work,
  case when a.financial_application_rk is not Null then 1 else 0 end as full_app,
  case when a.decision_dt is not Null then 1 else 0 end as considered,
  case when a.decision_approve_dt is not Null then 1 else 0 end as approve,
  case when a.utilization_dt is not Null then 1 else 0 end as utilization
  
from emart.short_applications_current b 
  inner join emart.financial_account_application a on b.financial_application_rk = a.financial_application_rk
where (not b.status in ('Дубль', 'Черный список'))
 and (a.financial_application_rk is not Null)
 and (b.phone_mobile is not Null)
 and create_dt >= '11Apr2017:0:0:0'dt 
 and create_dt <= '04May2017:0:0:0'dt 
;
quit;
'''

query = '''

alter table prod_ccall.full_app_201704 rename to user_kposminin.full_app_201704;


create table user_kposminin.id_cred_scor_test_201704 as
select 
     a.phone_mobile, 
     count(distinct m.id) as id_cnt,
     count(distinct acc.id) as acc_id_cnt,
     a.ymd as call_ymd,
     max(a.approve) as approve,
     max(a.utilization) as utilization, 
     max(a.in_work) as in_work, 
     a.utm_campaign, 
     max(a.considered) as considered,
     sum(acc.cnt) as cnt,
     sum(acc.visits_cnt) as visits_cnt, 
     sum(acc.hits) as hits,
     max(acc.max_score1) as max_score1,
     sum(acc.sum_score1) / sum(acc.cnt) as avg_score1,
     min(acc.min_score1) as min_score1,
     max(acc.max_score2) as max_score2,
     sum(acc.sum_score2) / sum(acc.cnt) as avg_score2,
     min(acc.min_score2) as min_score2,
     max(acc.max_score3) as max_score3,
     sum(acc.sum_score3) / sum(acc.cnt) as avg_score3,
     min(acc.min_score3) as min_score3,
     max(acc.max_score4) as max_score4,
     sum(acc.sum_score4) / sum(acc.cnt) as avg_score4,
     min(acc.min_score4) as min_score4,
     sum(acc.emailru) as emailru, 
     sum(acc.mobile_share * acc.cnt) / sum(acc.cnt) as mobile_share,
     sum(acc.vk_share * acc.cnt) / sum(acc.cnt) as vk_share,
     sum(acc.social_share * acc.cnt) / sum(acc.cnt) as social_share,
     sum(acc.work_hours_hits_share * acc.cnt) / sum(acc.cnt) as work_hours_hits_share,     
     (sum(acc.avg_hour_sum_sq) - power(sum(acc.avg_hour_sum), 2) ) / sum(acc.cnt) as hour_std,
     sum(acc.good_urlfr_share_score1 * acc.cnt) / sum(acc.cnt) as good_urlfr_share_score1,
     sum(acc.good_urlfr_share_score2 * acc.cnt) / sum(acc.cnt) as good_urlfr_share_score2,
     sum(acc.good_urlfr_share_score3 * acc.cnt) / sum(acc.cnt) as good_urlfr_share_score3,     
     max(trim(pc.provider)) as mob_provider,
     max(r.ind) as ind,
     max(r.pop_country_share) as pop_country_share, 
     max(r.pop_city_share) as pop_city_share, 
     max(r.population / r.area_sq_km) as density,
     max(r.area_sq_km) as area_sq_km,
     max(trim(r.federal_district)) as federal_district, 
     max(r.avg_salary_2015_rub) as avg_salary_2015_rub, 
     max(r.utc_time_zone_val) as utc_time_zone_val
     
  from
     user_kposminin.full_app_201704 a
     left join 
      (select 
         uid_str as id,
         property_value as phone_num,
         load_src
       from
         prod_dds.md_uid_property 
       where
         property_cd = 'PHONE'
      ) m on substr(m.phone_num,3,20) = substr(a.phone_mobile,2,20)

     left join user_kposminin.id_feat_accum_store acc on acc.id = m.id and acc.last_calc_ymd = date_add(a.ymd, -1)
     left join dds_dic.phone_codes pc on trim(pc.phone_code) = substr(a.phone_mobile,2,9)
     left join dds_dic.region_stat r on r.ind = pc.region_id
 group by
         a.phone_mobile,
         a.ymd,
         a.utm_campaign
;


'''

In [61]:
sc.stop()